In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Add the fourier_learning_ibm package to the path
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

['/home/jovyan',
 '/opt/conda/lib/python311.zip',
 '/opt/conda/lib/python3.11',
 '/opt/conda/lib/python3.11/lib-dynload',
 '',
 '/opt/conda/lib/python3.11/site-packages',
 '/home/jovyan/fourier_learning_ibm/']


In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg_graph import (
    HeisenbergModel,
    get_n_steps,
    get_graph,
    get_positions,
    get_initial_layout,
    get_prob0,
    extract_probs,
)
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import mthree
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [4]:
# Option1: Use IBM Quantum backend.
# Set up the Qiskit Runtime service (this is a one-time setup)
# QiskitRuntimeService.save_account(
#     token="YOUR_API_TOKEN",
#     channel="ibm_quantum",
# )

# Load saved credentials
service = QiskitRuntimeService()
# backend_qpu = service.least_busy(simulator=False, interactional=True)
backend_qpu = service.backend("ibm_marrakesh")

# Option2: Use local AerSimulator as the backend.
backend_sim = AerSimulator()

noise_model = NoiseModel.from_backend(backend_qpu)
backend_sim_noisy = AerSimulator(noise_model=noise_model)

print(f"Using backend QPU: {backend_qpu}")
print(f"Using backend simulator: {backend_sim}")
print(f"Using backend noisy simulator: {backend_sim_noisy}")

Using backend QPU: <IBMBackend('ibm_marrakesh')>
Using backend simulator: AerSimulator('aer_simulator')
Using backend noisy simulator: AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['id', 'sx', 'x', 'measure', 'reset', 'cz']>)


In [5]:
# Only for using Docker container
%cd fourier_learning_ibm/

/home/jovyan/fourier_learning_ibm


In [6]:
# Load parameters
# 'path' is the path created by the 'data_generate.ipynb'
path = f"data/2024-12-21T09:50Z"

# Load the graphs as a binary file
with open(f"{path}/params_object.pkl", "rb") as f:
    params_object = pickle.load(f)

graphs = params_object["graphs"]
all_Js = params_object["Js"]
all_expected_values = params_object["expected_values"]

# Load the parameters as a JSON file
with open(f"{path}/params_text.json", "r") as f:
    params = json.load(f)

n_samples = params["n_samples"]
n_qubits = params["n_qubits"]
graph_type = params["graph_type"]
backend_qpu_name = params["backend_qpu_name"]
beta = params["beta"]
C = params["C"]
n_features = params["n_features"]
times = params["times"]

for key, value in params.items():
    print(f"{key}: {value}")

n_samples: 56
n_qubits: 4
graph_type: line
backend_qpu_name: ibm_marrakesh
beta: 1
C: 5.274068563211588
n_features: 10
times: [0.0, 0.5956677688082146, 1.1913355376164292, 1.787003306424644, 2.3826710752328584, 2.9783388440410734, 3.574006612849288, 4.169674381657503, 4.765342150465717, 5.361009919273932]


# Create Fourier feature

## Trotter simulation (Exact)

In [7]:
# Exact simulation
data = []
probs_phase0_exact = {}
probs_phase1_exact = {}
probs_phase2_exact = {}
probs_phase3_exact = {}

# Generate the dataset with Fourier features
for i in range(n_samples):
    print(f"Calculating features for sample {i}/{n_samples}")
    Js = all_Js[i]
    lambda_ref = np.sum(Js)  # Reference eigenvalue.
    heisenberg_exact = HeisenbergModel(n_qubits, graphs[i])
    features_exact = []

    probs_phase0_exact[f"sample{i}"] = {}
    probs_phase1_exact[f"sample{i}"] = {}
    probs_phase2_exact[f"sample{i}"] = {}
    probs_phase3_exact[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        prob_phase0 = heisenberg_exact.exact_simulation(times[k], phase=0)
        prob_phase1 = heisenberg_exact.exact_simulation(times[k], phase=1)
        prob_phase2 = heisenberg_exact.exact_simulation(times[k], phase=2)
        prob_phase3 = heisenberg_exact.exact_simulation(times[k], phase=3)

        probs_phase0_exact[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_exact[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_exact[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_exact[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_ref * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features_exact.append(inner_product.real)
        if k != 0:
            features_exact.append(inner_product.imag)
    data.append([i, *features_exact, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_exact_df = pd.DataFrame(data, columns=columns)
display(data_exact_df)

# Save the exact data
data_exact_df.to_json(f"{path}/data_exact_df.json")
with open(f"{path}/probs_phase0_exact.json", "w") as f:
    json.dump(probs_phase0_exact, f)

with open(f"{path}/probs_phase1_exact.json", "w") as f:
    json.dump(probs_phase1_exact, f)

with open(f"{path}/probs_phase2_exact.json", "w") as f:
    json.dump(probs_phase2_exact, f)

with open(f"{path}/probs_phase3_exact.json", "w") as f:
    json.dump(probs_phase3_exact, f)

Calculating features for sample 0/56


/opt/conda/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/opt/conda/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


Calculating features for sample 1/56
Calculating features for sample 2/56
Calculating features for sample 3/56
Calculating features for sample 4/56
Calculating features for sample 5/56
Calculating features for sample 6/56
Calculating features for sample 7/56
Calculating features for sample 8/56
Calculating features for sample 9/56
Calculating features for sample 10/56
Calculating features for sample 11/56
Calculating features for sample 12/56
Calculating features for sample 13/56
Calculating features for sample 14/56
Calculating features for sample 15/56
Calculating features for sample 16/56
Calculating features for sample 17/56
Calculating features for sample 18/56
Calculating features for sample 19/56
Calculating features for sample 20/56
Calculating features for sample 21/56
Calculating features for sample 22/56
Calculating features for sample 23/56
Calculating features for sample 24/56
Calculating features for sample 25/56
Calculating features for sample 26/56
Calculating features 

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,expected_value
0,0,1.0,0.445222,-0.334450,-0.076516,-0.246695,0.260887,-0.236290,0.194003,-0.375524,...,-0.088496,-0.449425,0.310361,0.243236,0.181697,0.144211,0.012055,-0.234949,0.179659,2.653082
1,1,1.0,0.898467,-0.274751,0.652963,-0.505852,0.387604,-0.625680,0.193021,-0.582562,...,-0.409038,0.058431,-0.224129,0.100981,-0.145319,0.200972,-0.198100,0.295371,-0.317673,0.865078
2,2,1.0,0.229640,-0.334579,-0.033436,0.134404,-0.324680,-0.164133,-0.394101,0.197959,...,-0.358467,-0.250224,-0.665089,-0.130661,-0.032685,-0.234909,0.055842,-0.173877,0.806492,1.138950
3,3,1.0,0.582510,-0.291369,0.182650,-0.396080,0.297643,0.038127,0.409776,0.184381,...,-0.187041,0.444810,-0.184845,0.254132,-0.010350,0.506907,-0.085428,0.613365,-0.273668,4.367027
4,4,1.0,0.608345,0.525382,-0.101908,0.454631,-0.295540,-0.115162,0.236383,-0.515627,...,-0.290750,0.820802,0.286407,0.221878,0.556146,-0.232854,0.242910,-0.047304,-0.256219,4.566077
5,5,1.0,0.469406,0.177588,0.038280,0.352082,0.313165,-0.054052,0.396496,-0.201585,...,0.183209,0.255713,-0.031737,0.118090,-0.359962,0.640396,-0.119338,0.941076,0.053132,3.720380
6,6,1.0,0.686613,-0.426809,0.213101,-0.563756,0.103953,-0.314621,0.163761,-0.130313,...,-0.326888,-0.122486,-0.405228,-0.360965,-0.003736,-0.283770,0.403991,0.178785,0.412067,1.711405
7,7,1.0,0.625048,-0.323789,0.078057,-0.040346,0.044501,0.361204,0.191647,0.261326,...,-0.012245,-0.063806,-0.025632,0.215438,-0.143452,0.358147,-0.625835,0.008687,-0.841964,0.920606
8,8,1.0,0.380860,-0.333475,0.027449,0.051995,-0.192528,-0.264937,-0.783368,-0.108222,...,0.137889,-0.019845,-0.397631,-0.148212,-0.086051,0.307489,0.110217,-0.042837,0.092086,1.084233
9,9,1.0,0.631885,0.188391,0.028975,0.168200,-0.041906,0.125881,0.346376,0.303963,...,0.496888,-0.022547,0.314741,-0.485741,-0.132209,-0.389306,-0.290709,0.030485,-0.000724,3.526609


## Trotter simulation (Simulator, Noise free)

In [8]:
# Create circuits
# 1 jobs - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}
lambda_refs = {}

for i in range(n_samples):
    print(f"Creating circuits for sample {i}/{n_samples}")
    Js = all_Js[i]

    # lambda_ref はこの後も使うので、辞書として保存
    lambda_refs[f"sample{i}"] = np.sum(Js)  # Reference eigenvalue.

    heisenberg_sim = HeisenbergModel(n_qubits, graphs[i], backend=backend_sim)

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        n_steps = get_n_steps(times[k])
        circuit_phase0, exec_circuit_phase0 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=0)
        )
        circuit_phase1, exec_circuit_phase1 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=1)
        )
        circuit_phase2, exec_circuit_phase2 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=2)
        )
        circuit_phase3, exec_circuit_phase3 = (
            heisenberg_sim.get_trotter_simulation_circuit(times[k], n_steps, phase=3)
        )

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Creating circuits for sample 0/56
Creating circuits for sample 1/56
Creating circuits for sample 2/56
Creating circuits for sample 3/56
Creating circuits for sample 4/56
Creating circuits for sample 5/56
Creating circuits for sample 6/56
Creating circuits for sample 7/56
Creating circuits for sample 8/56
Creating circuits for sample 9/56
Creating circuits for sample 10/56
Creating circuits for sample 11/56
Creating circuits for sample 12/56
Creating circuits for sample 13/56
Creating circuits for sample 14/56
Creating circuits for sample 15/56
Creating circuits for sample 16/56
Creating circuits for sample 17/56
Creating circuits for sample 18/56
Creating circuits for sample 19/56
Creating circuits for sample 20/56
Creating circuits for sample 21/56
Creating circuits for sample 22/56
Creating circuits for sample 23/56
Creating circuits for sample 24/56
Creating circuits for sample 25/56
Creating circuits for sample 26/56
Creating circuits for sample 27/56
Creating circuits for sample 2

In [9]:
# Run jobs in batch

# For AerSimulator, we can't use job ids. Instead, we store the jobs in a list.
jobs = []
with Batch(backend=backend_sim):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/56
Running circuits for sample 1/56
Running circuits for sample 2/56
Running circuits for sample 3/56
Running circuits for sample 4/56
Running circuits for sample 5/56
Running circuits for sample 6/56
Running circuits for sample 7/56
Running circuits for sample 8/56
Running circuits for sample 9/56
Running circuits for sample 10/56
Running circuits for sample 11/56
Running circuits for sample 12/56
Running circuits for sample 13/56
Running circuits for sample 14/56
Running circuits for sample 15/56
Running circuits for sample 16/56
Running circuits for sample 17/56
Running circuits for sample 18/56
Running circuits for sample 19/56
Running circuits for sample 20/56
Running circuits for sample 21/56
Running circuits for sample 22/56
Running circuits for sample 23/56
Running circuits for sample 24/56
Running circuits for sample 25/56
Running circuits for sample 26/56
Running circuits for sample 27/56
Running circuits for sample 28/56
Running circuits for sam

In [10]:
check_list = []
for i in range(n_samples):
    job_id = jobs[i].job_id()
    check_list.append(
        {
            "Job ID": job_id,
            "sample_id": i,
            "Status": job.status().name,
        }
    )

check_df = pd.DataFrame(check_list)
display(check_df)

,Job ID,sample_id,Status
0,50c83319-9196-4927-ae79-624d76065783,0,DONE
1,af4e65b4-ab31-4519-9627-93d9af80040c,1,DONE
2,ae3f2851-0e96-4dda-b119-6205856d8c14,2,DONE
3,f865ee70-24ff-4b14-ac86-b31bc3435de5,3,DONE
4,2c8fad97-4ffd-437f-8e1a-e034c79f3de1,4,DONE
5,84f89ca7-504f-4391-9cdb-d5175d1b9f7e,5,DONE
6,bd0271d4-2556-4246-af6a-4bb8119f2ac6,6,DONE
7,161afdef-ef1f-4a41-bd62-d6a96df30ce2,7,DONE
8,a150f8b0-8d62-42ad-96ec-030d1367c1ce,8,DONE
9,34577099-128e-4f78-92ee-30ab330e4733,9,DONE


In [11]:
# Post-process
data_sim = []
probs_phase0_sim = {}
probs_phase1_sim = {}
probs_phase2_sim = {}
probs_phase3_sim = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim[f"sample{i}"] = {}
    probs_phase1_sim[f"sample{i}"] = {}
    probs_phase2_sim[f"sample{i}"] = {}
    probs_phase3_sim[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_sim_df = pd.DataFrame(data_sim, columns=columns)
display(data_sim_df)

# Save the simulation data
data_sim_df.to_json(f"{path}/data_sim_df.json")
with open(f"{path}/probs_phase0_sim.json", "w") as f:
    json.dump(probs_phase0_sim, f)

with open(f"{path}/probs_phase1_sim.json", "w") as f:
    json.dump(probs_phase1_sim, f)

with open(f"{path}/probs_phase2_sim.json", "w") as f:
    json.dump(probs_phase2_sim, f)

with open(f"{path}/probs_phase3_sim.json", "w") as f:
    json.dump(probs_phase3_sim, f)

 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No counts for |0...0> state
 > No co

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,expected_value
0,0,1.0,0.452016,-0.250536,-0.140064,-0.281868,-0.087767,-0.539257,0.229866,-0.393859,...,-0.055623,-0.593693,0.339372,-0.302407,0.106476,0.155672,-0.218801,0.447846,0.022096,2.653082
1,1,1.0,0.898650,-0.263049,0.675833,-0.475762,0.328420,-0.598678,0.206833,-0.570881,...,-0.395448,0.114037,-0.150381,0.121032,-0.088348,0.215246,-0.132656,0.324917,-0.258524,0.865078
2,2,1.0,0.255602,-0.337788,-0.030845,0.118532,-0.398410,-0.113194,-0.422474,0.146621,...,-0.428185,-0.152694,-0.676709,-0.138684,-0.028195,-0.175973,0.230144,-0.197841,0.774496,1.138950
3,3,1.0,0.557356,-0.145905,0.045288,-0.362480,0.196620,-0.120540,0.379264,0.506354,...,0.653179,0.073344,-0.103623,0.428477,0.094425,0.388704,0.319345,-0.015781,-0.051812,4.367027
4,4,1.0,0.648849,0.494100,-0.064274,0.444597,-0.307607,0.030677,0.137188,-0.419738,...,-0.322561,0.918773,0.021295,0.440178,0.425765,-0.035345,0.444720,-0.214085,0.084410,4.566077
5,5,1.0,0.411462,0.074202,-0.068798,0.313729,0.281338,0.269611,0.385650,-0.456242,...,-0.561244,0.016111,0.026795,0.303487,-0.437017,0.288236,-0.661586,0.200012,-0.054460,3.720380
6,6,1.0,0.694964,-0.335801,0.234051,-0.499954,0.105421,-0.451840,0.147450,-0.067116,...,0.125300,-0.283869,-0.164969,-0.247804,-0.059724,-0.213618,0.381579,-0.321810,0.627993,1.711405
7,7,1.0,0.622788,-0.340083,0.067842,-0.072870,0.038118,0.355026,0.179545,0.281087,...,-0.021163,-0.073758,-0.023628,0.237000,-0.169254,0.396796,-0.631461,-0.012685,-0.833987,0.920606
8,8,1.0,0.374676,-0.322285,0.026558,0.090380,-0.163263,-0.298393,-0.784260,-0.138186,...,0.178754,-0.014492,-0.402910,-0.158241,-0.074523,0.331899,0.089681,-0.052559,0.077163,1.084233
9,9,1.0,0.611245,0.137231,0.028422,0.103966,-0.069124,0.205660,0.289778,0.343693,...,0.529171,0.112941,0.264273,-0.458552,-0.112556,-0.467851,-0.292308,-0.080452,0.035895,3.526609


## Trotter simulation (Simulator, Noisy)

In [12]:
# Create circuits
# 1 jobs - 1 sample
circuits_phase0 = {}
circuits_phase1 = {}
circuits_phase2 = {}
circuits_phase3 = {}
exec_circuits_phase0 = {}
exec_circuits_phase1 = {}
exec_circuits_phase2 = {}
exec_circuits_phase3 = {}

for i in range(n_samples):
    print(f"Creating circuits for sample {i}/{n_samples}")
    Js = all_Js[i]
    heisenberg_sim_noisy = HeisenbergModel(
        n_qubits, graphs[i], backend=backend_sim_noisy
    )

    circuits_phase0[f"sample{i}"] = {}
    circuits_phase1[f"sample{i}"] = {}
    circuits_phase2[f"sample{i}"] = {}
    circuits_phase3[f"sample{i}"] = {}
    exec_circuits_phase0[f"sample{i}"] = {}
    exec_circuits_phase1[f"sample{i}"] = {}
    exec_circuits_phase2[f"sample{i}"] = {}
    exec_circuits_phase3[f"sample{i}"] = {}
    for k in range(n_features):
        n_steps = get_n_steps(times[k])
        circuit_phase0, exec_circuit_phase0 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=0
            )
        )
        circuit_phase1, exec_circuit_phase1 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=1
            )
        )
        circuit_phase2, exec_circuit_phase2 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=2
            )
        )
        circuit_phase3, exec_circuit_phase3 = (
            heisenberg_sim_noisy.get_trotter_simulation_circuit(
                times[k], n_steps, phase=3
            )
        )

        circuits_phase0[f"sample{i}"][f"f_{k}"] = circuit_phase0
        circuits_phase1[f"sample{i}"][f"f_{k}"] = circuit_phase1
        circuits_phase2[f"sample{i}"][f"f_{k}"] = circuit_phase2
        circuits_phase3[f"sample{i}"][f"f_{k}"] = circuit_phase3
        exec_circuits_phase0[f"sample{i}"][f"f_{k}"] = exec_circuit_phase0
        exec_circuits_phase1[f"sample{i}"][f"f_{k}"] = exec_circuit_phase1
        exec_circuits_phase2[f"sample{i}"][f"f_{k}"] = exec_circuit_phase2
        exec_circuits_phase3[f"sample{i}"][f"f_{k}"] = exec_circuit_phase3

Creating circuits for sample 0/56
Creating circuits for sample 1/56
Creating circuits for sample 2/56
Creating circuits for sample 3/56
Creating circuits for sample 4/56
Creating circuits for sample 5/56
Creating circuits for sample 6/56
Creating circuits for sample 7/56
Creating circuits for sample 8/56
Creating circuits for sample 9/56
Creating circuits for sample 10/56
Creating circuits for sample 11/56
Creating circuits for sample 12/56
Creating circuits for sample 13/56
Creating circuits for sample 14/56
Creating circuits for sample 15/56
Creating circuits for sample 16/56
Creating circuits for sample 17/56
Creating circuits for sample 18/56
Creating circuits for sample 19/56
Creating circuits for sample 20/56
Creating circuits for sample 21/56
Creating circuits for sample 22/56
Creating circuits for sample 23/56
Creating circuits for sample 24/56
Creating circuits for sample 25/56
Creating circuits for sample 26/56
Creating circuits for sample 27/56
Creating circuits for sample 2

In [13]:
# Run jobs in batch
jobs = []  # For AerSimulator, we can't use job ids.
with Batch(backend=backend_sim_noisy):
    sampler = Sampler()

    for i in range(n_samples):
        print(f"Running circuits for sample {i}/{n_samples}")
        exec_circuits_per_sample = []
        exec_circuits_per_sample += [
            exec_circuits_phase0[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase1[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase2[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]
        exec_circuits_per_sample += [
            exec_circuits_phase3[f"sample{i}"][f"f_{k}"] for k in range(n_features)
        ]

        job = sampler.run(exec_circuits_per_sample)
        jobs.append(job)

Running circuits for sample 0/56
Running circuits for sample 1/56
Running circuits for sample 2/56
Running circuits for sample 3/56
Running circuits for sample 4/56
Running circuits for sample 5/56
Running circuits for sample 6/56
Running circuits for sample 7/56
Running circuits for sample 8/56
Running circuits for sample 9/56
Running circuits for sample 10/56
Running circuits for sample 11/56
Running circuits for sample 12/56
Running circuits for sample 13/56
Running circuits for sample 14/56
Running circuits for sample 15/56
Running circuits for sample 16/56
Running circuits for sample 17/56
Running circuits for sample 18/56
Running circuits for sample 19/56
Running circuits for sample 20/56
Running circuits for sample 21/56
Running circuits for sample 22/56
Running circuits for sample 23/56
Running circuits for sample 24/56
Running circuits for sample 25/56
Running circuits for sample 26/56
Running circuits for sample 27/56
Running circuits for sample 28/56
Running circuits for sam

In [14]:
check_list = []
for i in range(n_samples):
    job_id = jobs[i].job_id()
    check_list.append(
        {
            "Job ID": job_id,
            "sample_id": i,
            "Status": job.status().name,
        }
    )

check_df = pd.DataFrame(check_list)
display(check_df)

,Job ID,sample_id,Status
0,dda4cbd8-1907-4f86-a525-21c00ee5a8a9,0,DONE
1,b5a47fdb-9b1e-4449-a1f7-a74c36247c46,1,DONE
2,ee20d17a-f401-4795-aa3a-3d77ce2af662,2,DONE
3,ab73285d-4fbb-42fe-bb40-dfae906bf1ec,3,DONE
4,954fe2c3-2bfb-4bce-adf7-c511531f2761,4,DONE
5,144111b7-e871-451a-a461-5678299761c8,5,DONE
6,2a6b1446-aa41-4b0f-b563-7dfe29f1af0c,6,DONE
7,a97fc745-6918-4657-b3ce-4958aa251b0a,7,DONE
8,f8eb632f-686f-4767-9b66-7f3c43fbe6b2,8,DONE
9,f1721ed8-f854-48dd-aa04-4b5f925e0fee,9,DONE


In [15]:
# Post-process
data_sim_noisy = []
probs_phase0_sim_noisy = {}
probs_phase1_sim_noisy = {}
probs_phase2_sim_noisy = {}
probs_phase3_sim_noisy = {}

for i in range(n_samples):
    features = []
    probs_phase0_sim_noisy[f"sample{i}"] = {}
    probs_phase1_sim_noisy[f"sample{i}"] = {}
    probs_phase2_sim_noisy[f"sample{i}"] = {}
    probs_phase3_sim_noisy[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        # Get results of each phase in a batch
        results_phase0 = jobs[i].result()[:n_features]
        results_phase1 = jobs[i].result()[n_features : 2 * n_features]
        results_phase2 = jobs[i].result()[2 * n_features : 3 * n_features]
        results_phase3 = jobs[i].result()[3 * n_features :]

        prob_phase0 = get_prob0(results_phase0[k], n_qubits)
        prob_phase1 = get_prob0(results_phase1[k], n_qubits)
        prob_phase2 = get_prob0(results_phase2[k], n_qubits)
        prob_phase3 = get_prob0(results_phase3[k], n_qubits)

        probs_phase0_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_sim_noisy[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_refs[f"sample{i}"] * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )
        features.append(inner_product.real)
        if k != 0:
            features.append(inner_product.imag)
    data_sim_noisy.append([i, *features, all_expected_values[i]])

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
data_sim_noisy_df = pd.DataFrame(data_sim_noisy, columns=columns)
display(data_sim_noisy_df)

# Save the simulation data
data_sim_noisy_df.to_json(f"{path}/data_sim_noisy_df.json")
with open(f"{path}/probs_phase0_sim_noisy.json", "w") as f:
    json.dump(probs_phase0_sim_noisy, f)

with open(f"{path}/probs_phase1_sim_noisy.json", "w") as f:
    json.dump(probs_phase1_sim_noisy, f)

with open(f"{path}/probs_phase2_sim_noisy.json", "w") as f:
    json.dump(probs_phase2_sim_noisy, f)

with open(f"{path}/probs_phase3_sim_noisy.json", "w") as f:
    json.dump(probs_phase3_sim_noisy, f)

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_5 Im,f_6 Re,f_6 Im,f_7 Re,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,expected_value
0,0,0.980469,0.414519,-0.196669,-0.124413,-0.251420,-0.115187,-0.477212,0.261527,-0.341515,...,-0.059563,-0.495661,0.297585,-0.220554,0.049595,0.112642,-0.178164,0.368294,0.036212,2.653082
1,1,0.979492,0.856371,-0.223990,0.615062,-0.411922,0.342600,-0.528439,0.192524,-0.496673,...,-0.343620,0.100595,-0.162242,0.112704,-0.060585,0.166130,-0.092151,0.283934,-0.243153,0.865078
2,2,0.967773,0.230937,-0.304498,-0.039656,0.116682,-0.312337,-0.088457,-0.309474,0.133238,...,-0.415614,-0.143058,-0.596920,-0.112214,-0.056922,-0.138375,0.156043,-0.181719,0.620830,1.138950
3,3,0.972656,0.496837,-0.140253,0.080979,-0.291592,0.192619,-0.096377,0.350416,0.462852,...,0.530921,0.088786,-0.085938,0.381995,0.089599,0.339086,0.266363,0.019513,-0.017760,4.367027
4,4,0.976562,0.589439,0.476121,-0.113240,0.407916,-0.300273,0.038279,0.103804,-0.330826,...,-0.296849,0.787106,0.003784,0.350046,0.338345,-0.034333,0.340690,-0.196547,0.100080,4.566077
5,5,0.977539,0.389773,0.054507,-0.038356,0.334336,0.234748,0.227540,0.346831,-0.389973,...,-0.499162,0.008418,0.035193,0.300055,-0.346551,0.222863,-0.521291,0.151298,-0.057798,3.720380
6,6,0.978516,0.647034,-0.303691,0.186511,-0.507961,0.077373,-0.409271,0.136008,-0.047763,...,0.127396,-0.241049,-0.171899,-0.168334,-0.030514,-0.208153,0.285213,-0.238295,0.549531,1.711405
7,7,0.975586,0.587036,-0.319863,0.079819,-0.054795,0.010833,0.361816,0.179669,0.227554,...,-0.014193,-0.070391,-0.051900,0.163325,-0.103416,0.317175,-0.503705,0.023454,-0.690301,0.920606
8,8,0.989258,0.334213,-0.350994,0.046408,0.024349,-0.165207,-0.232130,-0.716139,-0.080792,...,0.168093,-0.014027,-0.344188,-0.108922,-0.066990,0.254931,0.121852,-0.039389,0.045744,1.084233
9,9,0.976562,0.592646,0.096750,0.023824,0.139924,-0.071649,0.157518,0.255265,0.314008,...,0.440585,0.054557,0.230475,-0.360719,-0.093596,-0.418284,-0.201552,-0.049808,0.018417,3.526609
